In [30]:
import tensorflow as tf
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
import shutil
import sklearn
import cv2
import csv
import math
#import image
from skimage.io import imread
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.image as mpimg
#导入图表库
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import applications
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import normalize

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19

from keras.applications.vgg19 import preprocess_input

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing   

from sklearn.model_selection import KFold
from keras.utils import plot_model
# PCA 主成分分析
from sklearn.decomposition import PCA
from keras.callbacks import TensorBoard
print("hello tensorflow")

hello tensorflow


## 读取数据

In [31]:
# 读取数据
X = pd.read_csv(open("../../emotionData/finalData.csv",encoding='utf-8'))
#X = pd.DataFrame(X)
X = X[0:239999]
X = X.values

print(X.shape)
print(X)

(239999, 1024)
[[0.31910929 0.52602024 0.47184321 ... 0.39789196 0.36649874 0.35365854]
 [0.31910929 0.52602024 0.47184321 ... 0.08695652 0.08312343 0.0804878 ]
 [0.31910929 0.52602024 0.47184321 ... 0.39262187 0.395466   0.39878049]
 ...
 [0.64525379 0.91699595 0.65415632 ... 0.47694335 0.59823678 0.60487805]
 [0.55993044 0.40413241 0.47403153 ... 0.75625824 0.69269521 0.59512195]
 [0.64658885 0.30753055 0.35559861 ... 0.50988142 0.5302267  0.47560976]]


In [85]:
with open('../../emotionData/s25w-r-n.CSV',"rt", encoding="utf-8") as vsvfile:
    reader = csv.reader(vsvfile)
    y = [row[128] for row in reader]
#y = pd.DataFrame(y)
y = y[1:]
y = y[0:239999]
print(y)

['0.136147', '0.058372', '0.126422', '0.108744', '0.110896', '0.17498', '0.134798', '0.133093', '0.114068', '0.08394', '0.138001', '0.162281', '0.024871', '0.049312', '0.074712', '0.027434', '0.127124', '0.051743', '0.084803', '0.0248', '0.105067', '0.141144', '0.08615', '0.135191', '0.152599', '0.168492', '0.074377', '0.190048', '0.03048', '0.059181', '0.169326', '0.141099', '0.140078', '0.123199', '0.123199', '0.118113', '0.114252', '0.118795', '0.086571', '0.029933', '0.075203', '0.084097', '0.160213', '0.178622', '0.192187', '0.188378', '0.946881', '0.102354', '0.167487', '0.183242', '0.123613', '0.162617', '0.016962', '0.056552', '0.199944', '0.025987', '0.024746', '0.119984', '0.071054', '0.142389', '0.018371', '0.021518', '0.149701', '0.159314', '0.099027', '0.119263', '0.029829', '0.02363', '0.018654', '0.094959', '0.027948', '0.025592', '0.029134', '0.028091', '0.053663', '0.026662', '0.054723', '0.043873', '0.117656', '0.025713', '0.101948', '0.067571', '0.030815', '0.030815'

In [86]:
# with open('../../emotionData/s25w-r-n.CSV',"rt", encoding="utf-8") as vsvfile:
#     reader = csv.reader(vsvfile)
#     y = [row[128] for row in reader]
# #y = pd.DataFrame(y)
# y = y[1:]
# y = y[0:239999]



my_new_list = []
for i in y:
    my_new_list.append(float(i) * 2)


y = pd.DataFrame(my_new_list)
print(y.shape)
print(y)

(239999, 1)
               0
0       0.272294
1       0.116744
2       0.252844
3       0.217488
4       0.221792
5       0.349960
6       0.269596
7       0.266186
8       0.228136
9       0.167880
10      0.276002
11      0.324562
12      0.049742
13      0.098624
14      0.149424
15      0.054868
16      0.254248
17      0.103486
18      0.169606
19      0.049600
20      0.210134
21      0.282288
22      0.172300
23      0.270382
24      0.305198
25      0.336984
26      0.148754
27      0.380096
28      0.060960
29      0.118362
...          ...
239969  0.115402
239970  0.110400
239971  0.163564
239972  0.142312
239973  0.117764
239974  0.149630
239975  0.226224
239976  0.101030
239977  0.289610
239978  0.251132
239979  0.255518
239980  0.286282
239981  0.216160
239982  0.199412
239983  0.249674
239984  0.238512
239985  0.187892
239986  0.181140
239987  0.299420
239988  0.272436
239989  0.205890
239990  0.255002
239991  0.143532
239992  0.146736
239993  0.289078
239994  0.202208
23

In [87]:
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Dense(1024, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(1024, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))


model.add(Dense(512, input_dim=1024, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(512, input_dim=512, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(256, input_dim=512, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(256, input_dim=256, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(128, input_dim=256, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(128, input_dim=128, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(64, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(64, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(32, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(16, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(16, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(8, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))

model.add(Dense(8, activation='relu'))
model.add(keras.layers.normalization.BatchNormalization(axis=-1, 
                                              momentum=0.99, 
                                              epsilon=0.001, 
                                              center=True, 
                                              scale=True, 
                                              beta_initializer='zeros', 
                                              gamma_initializer='ones',
                                              ))


model.add(Dense(1, activation='sigmoid'))

In [88]:
from keras import backend as K

from sklearn import metrics

def mre(y_true, y_pred):
    return K.mean(K.sum(K.abs(100*K.ones_like(y_true) * (y_true - y_pred) / y_true)))


def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

def r_square_final(y_true, y_pred):
    SSR = K.mean(K.square(y_pred-K.mean(y_true)),axis=-1)
    SST = K.mean(K.square(y_true-K.mean(y_true)),axis=-1)
    return SSR/SST

In [89]:
model.compile(
    loss='mean_squared_error', 
    optimizer='adam',
    metrics=['mean_squared_error',
             'mean_absolute_error',
             'mean_absolute_percentage_error',
             'mean_squared_logarithmic_error',
             r_square_final,
            # metrics.r2_score,
            # metrics.r2_score,
             pearson_r,
             mre]
)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_76 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_82 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_77 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_83 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_78 (Batc (None, 512)               2048      
_________________________________________________________________
dense_84 (Dense)             (None, 512)               262656    
__________

In [90]:
callbacks_list = [
    # 训练可视化
    keras.callbacks.TensorBoard(
        log_dir='./fit_Euclidean_Distance_logs',  # log 目录
        histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
        #batch_size=32,     # 用多大量的数据计算直方图
        write_graph=True,  # 是否存储网络结构图
        write_grads=True, # 是否可视化梯度直方图
        write_images=True,# 是否可视化参数
        embeddings_freq=0,
        embeddings_layer_names=None, 
        embeddings_metadata=None),
    
    # 训练自动更改学习率
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_mean_squared_error', 
        factor=0.2,
        patience=5,
        verbose=1,
        mode='min',
        min_delta=0.0001, 
        cooldown=0, 
        min_lr=0.0001),
    
    # 当被监测的数量不再提升，则停止训练
    keras.callbacks.EarlyStopping(
        monitor='val_mean_squared_error', 
        min_delta=0, 
        patience=30, 
        verbose=1, 
        mode='min', 
        baseline=None, 
        restore_best_weights=False)

]

In [91]:
import keras.backend.tensorflow_backend as KTF #进行配置，每个GPU使用80%上限现存


#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用编号为0号的GPU

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在80%以内
session = tf.Session(config=config) # 设置session KTF.set_session(session )
#设置session
KTF.set_session(session )

#注意：
#config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在80%以内
#设置GPU使用上限，可以有效避免显存不足而导致的jupyter 问题:
#The kernel appears to have died. It will restart automatically

In [92]:
keras.backend.get_session().run(tf.global_variables_initializer())


his=model.fit(
    X,
    y,
    
    validation_split=0.1,
    epochs=70,
    batch_size=1024,
    
    shuffle=True,
    verbose=1,
    
    
    callbacks=callbacks_list
)

Train on 215999 samples, validate on 24000 samples
Epoch 1/70
215999/215999 [==============================] - 55s 254us/step - loss: 0.0540 - mean_squared_error: 0.0540 - mean_absolute_error: 0.1928 - mean_absolute_percentage_error: 1983.5310 - mean_squared_logarithmic_error: 0.0284 - r_square_final: 200654.0407 - pearson_r: 0.0018 - mre: inf - val_loss: 0.0123 - val_mean_squared_error: 0.0123 - val_mean_absolute_error: 0.0962 - val_mean_absolute_percentage_error: 61.0781 - val_mean_squared_logarithmic_error: 0.0082 - val_r_square_final: 68802427.5936 - val_pearson_r: 0.0044 - val_mre: 61959.4569
Epoch 2/70
215999/215999 [==============================] - 51s 238us/step - loss: 0.0165 - mean_squared_error: 0.0165 - mean_absolute_error: 0.0717 - mean_absolute_percentage_error: 1293.0853 - mean_squared_logarithmic_error: 0.0071 - r_square_final: 27037.5976 - pearson_r: 0.0105 - mre: inf - val_loss: 0.0041 - val_mean_squared_error: 0.0041 - val_mean_absolute_error: 0.0528 - val_mean_abso

215999/215999 [==============================] - 51s 237us/step - loss: 0.0149 - mean_squared_error: 0.0149 - mean_absolute_error: 0.0607 - mean_absolute_percentage_error: 1053.9870 - mean_squared_logarithmic_error: 0.0060 - r_square_final: 65293.2693 - pearson_r: 0.0619 - mre: inf - val_loss: 0.0039 - val_mean_squared_error: 0.0039 - val_mean_absolute_error: 0.0515 - val_mean_absolute_percentage_error: 32.0684 - val_mean_squared_logarithmic_error: 0.0027 - val_r_square_final: 4134051.1710 - val_pearson_r: -0.0071 - val_mre: 32524.3347
Epoch 16/70
215999/215999 [==============================] - 51s 237us/step - loss: 0.0149 - mean_squared_error: 0.0149 - mean_absolute_error: 0.0607 - mean_absolute_percentage_error: 1070.2193 - mean_squared_logarithmic_error: 0.0060 - r_square_final: 11831.5390 - pearson_r: 0.0711 - mre: inf - val_loss: 0.0039 - val_mean_squared_error: 0.0039 - val_mean_absolute_error: 0.0517 - val_mean_absolute_percentage_error: 32.1731 - val_mean_squared_logarithmic_

Epoch 30/70
215999/215999 [==============================] - 50s 234us/step - loss: 0.0140 - mean_squared_error: 0.0140 - mean_absolute_error: 0.0610 - mean_absolute_percentage_error: 999.2960 - mean_squared_logarithmic_error: 0.0057 - r_square_final: 102526.7752 - pearson_r: 0.2495 - mre: inf - val_loss: 0.0041 - val_mean_squared_error: 0.0041 - val_mean_absolute_error: 0.0518 - val_mean_absolute_percentage_error: 31.8340 - val_mean_squared_logarithmic_error: 0.0028 - val_r_square_final: 3771275.9619 - val_pearson_r: 0.0032 - val_mre: 32279.8796
Epoch 31/70
215999/215999 [==============================] - 51s 234us/step - loss: 0.0139 - mean_squared_error: 0.0139 - mean_absolute_error: 0.0610 - mean_absolute_percentage_error: 985.2876 - mean_squared_logarithmic_error: 0.0057 - r_square_final: 18425.6673 - pearson_r: 0.2632 - mre: inf - val_loss: 0.0043 - val_mean_squared_error: 0.0043 - val_mean_absolute_error: 0.0522 - val_mean_absolute_percentage_error: 32.0162 - val_mean_squared_lo

In [93]:
# calculate predictions

test_X = X
#actual = y

# from keras.models import load_model

# model = load_model('model.h5')
predictions = model.predict(test_X)

In [94]:
# round predictions
B = []
for x in predictions:
    for i in x:
        B.append(i)
        
B = B[1:239999]

In [95]:
actual = y[1:239999]

In [96]:
sum, sum2 = float(0), float(0)
max, mre, res, abres = float(0), float(0), float(0), float(0)

zipped = zip(actual, B)

In [97]:
print("Actual\t\tPredicted\t MRE\t\tRes. Error\tAbs. Res. Error")
for i, j in zipped:
    mre = float(abs(float(i) - float(j))) / float(i)
    res = float(i) - float(j)
    abres = float(abs(res))
    sum = sum + mre
    sum2 = sum2 + res
    if (abs(float(i) - float(j)) / float(i) > max):
        max = float(abs(float(i) - float(j))) / float(i)
    stringI = '{:.4f}'.format(float(i))
    stringJ = '{:.4f}'.format(float(j))
    #print (stringI, "\t\t", stringJ, "\t", '{:.4f}'.format(mre), "\t", '{:.4f}'.format(res), "\t", '{:.4f}'.format(abres))

Actual		Predicted	 MRE		Res. Error	Abs. Res. Error


ZeroDivisionError: float division by zero

In [98]:
def mean_squared_error_byself(y_true,y_predict):
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    return np.sum((y_true - y_predict) ** 2 ) / len(y_true)

def root_mean_squared_error_byself(y_true,y_predict):
    """计算y_true和y_predict之间的RMES"""
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    return math.sqrt(np.sum((y_true - y_predict) ** 2 ) / len(y_true))

def mean_absolute_error_byself(y_true,y_predict):
    """计算y_true和y_predict之间的MAE"""
    assert len(y_true) == len(y_predict), \
        "the size of y_true must be equal to be the size of y_predict"
    print(len(y_true))
    print(len(y_predict))
    
    abselute_value = abs(y_true - y_predict)
    return ((np.sum(float(abs(y_true - y_predict)))) / float(len(y_true)))

print("Mean Absolute Error      ", mean_absolute_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Squared Error       ", mean_squared_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Root Mean Square Error : ", root_mean_squared_error_byself(np.array(actual, dtype = float) , np.array(B, dtype = float)))


239998
239998


MemoryError: 

In [99]:
print("\nMean Magnitude of Relative Error (MMRE) : ", sum / len(actual))
print("MaxMRE :                 ", max)
print("Explained Variance Score ", metrics.explained_variance_score(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Absolute Error      ", metrics.mean_absolute_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Mean Squared Error       ", metrics.mean_squared_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Median Absolute Error    ", metrics.median_absolute_error(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("R-Square Score           ", metrics.r2_score(np.array(actual, dtype = float) , np.array(B, dtype = float)))
print("Root Mean Square Error : ", (sum2 / len(B)) ** (1 / 2))


Mean Magnitude of Relative Error (MMRE) :  0.0
MaxMRE :                  0.0
Explained Variance Score  0.1219979492413863
Mean Absolute Error       0.05999628453945569
Mean Squared Error        0.012182524843395954
Median Absolute Error     0.0471940476474762
R-Square Score            0.12146756428692562
Root Mean Square Error :  0.0
